# Schnorr

In this chapter we prove knowledge of a scalar using the Schnorr identification protocol.

# Jupyter setup

Run the following snippet to set up your jupyter notebook for the workshop.

In [ ]:
import sys

# Add project root so we can import local modules
root_dir = sys.path.append("..")
sys.path.append(root_dir)
    
# Import here so cells don't depend on each other
from IPython.display import display
import ipywidgets as widgets
import random

from typing import List, Tuple
from local.ec.static import Scalar, CurvePoint, ONE_POINT
import local.stats as stats

In [ ]:
# You can adjust the selection any time

matching_dropdown = widgets.Dropdown(
    options=[
        ("Matching secret key (honest Peggy 😇)", True),
        ("Different secret key (lying Peggy 😈)", False)],
    value=True,
    description="Scenario:",
)
matching_dropdown

In [ ]:
x = Scalar.random()
if matching_dropdown.value:
    P = ONE_POINT * x
else:
    P = ONE_POINT * Scalar.random()

print(f"One * {x} = {P}")

In [ ]:
class Peggy:
    def __init__(self, x: Scalar):
        self.x = x
        
    def commit(self) -> CurvePoint:
        self.r = Scalar.random()
        R = ONE_POINT * self.r
        return R
    
    def respond(self, e: Scalar) -> Scalar:
        s = self.r + e * self.x
        return s

class Victor:
    def __init__(self, P: CurvePoint):
        self.P = P
    
    def challenge(self, R: CurvePoint) -> Scalar:
        self.R = R
        self.e = Scalar.random()
        return self.e
    
    def verify(self, s: Scalar) -> bool:
        return s * ONE_POINT == self.R + e * self.P

In [ ]:
# Feel free to run this multiple times

peggy = Peggy(x)
victor = Victor(P)

R = peggy.commit()
print(f"R = {R}")

e = victor.challenge(R)
print(f"e = {e}")

s = peggy.respond(e)
print(f"s = {s}")
print()

# Victor is convinced
if victor.verify(s):
    # Matching secret key (good)
    if matching_dropdown.value:
        print("Convinced 👌 (expected)")
    # Different secret key (evil)
    else:
        print("Convinced 👌 (Victor was fooled)")
# Victor is not convinced
else:
    # Matching secret key (good)
    if matching_dropdown.value:
        print("Not convinced... 🤨 (Peggy was dumb)")
    # Different secret key (evil)
    else:
        print("Not convinced... 🤨 (expected)")

In [ ]:
n_exchanges_complete_slider = widgets.IntSlider(min=10, max=1000, value=10, step=10, description="#Exchanges")
n_exchanges_complete_slider

In [ ]:
# Honest case
x2 = Scalar.random()
P2 = ONE_POINT * x2

honest_peggy = Peggy(x2)
victor = Victor(P2)

peggy_success = 0

for _ in range(n_exchanges_complete_slider.value):
    R = honest_peggy.commit()
    e = victor.challenge(R)
    s = honest_peggy.respond(e)

    if victor.verify(s):
        peggy_success += 1
        
peggy_success_rate = peggy_success / n_exchanges_complete_slider.value * 100

print(f"Running {n_exchanges_complete_slider.value} exchanges")
print(f"Honest Peggy wins {peggy_success_rate:0.2f}% of the time")
print()

assert peggy_success_rate == 100
print("Peggy always wins if she is honest")

In [ ]:
n_exchanges_sound_slider = widgets.IntSlider(min=10, max=1000, value=10, step=10, description="#Exchanges")
n_exchanges_sound_slider

In [ ]:
# Lying case
x3 = Scalar.random()
P3 = ONE_POINT * Scalar.random()

lying_peggy = Peggy(x3)
victor = Victor(P3)

victor_success = 0

for _ in range(n_exchanges_sound_slider.value):
    R = lying_peggy.commit()
    e = victor.challenge(R)
    s = lying_peggy.respond(e)

    if not victor.verify(s):
        victor_success += 1

victor_success_rate = victor_success / n_exchanges_sound_slider.value * 100

print(f"Running {n_exchanges_sound_slider.value} exchanges")
print(f"Victor wins against lying Peggy {victor_success_rate:0.2f}% of the time")
print()

if victor_success_rate < 75:
    print("Victor has a small chance of getting fooled which decreases with the curve size")
else:
    print("It is basically impossible to fool Victor")

In [ ]:
n_transcripts_slider = widgets.IntSlider(min=1000, max=50000, value=10000, step=1000, description="#Transcripts")
n_transcripts_slider

In [ ]:
peggy = Peggy(x)
victor = Victor(P)

def real_transcript() -> Tuple:
    R = peggy.commit()
    e = victor.challenge(R)
    s = peggy.respond(e)
    return (R, e, s)


def fake_transcript() -> Tuple:
    e = Scalar.random()
    s = Scalar.random()
    R = ONE_POINT * s - P * e
    return (R, e, s)


print("Real transcript: {}".format(real_transcript()))
print("Fake transcript: {}".format(fake_transcript()))
print()

real_samples = [real_transcript() for _ in range(n_transcripts_slider.value)]
fake_samples = [fake_transcript() for _ in range(n_transcripts_slider.value)]

# The chi-square test is only valid if most bins are filled
# Increase the number of transcripts if there are too many empty bins

null_hypothesis = stats.chi_square_equal(real_samples, fake_samples)
print()

if null_hypothesis:
    print("Real and fake transcripts are the same distribution.")
    print("Victor learns nothing 👌")
else:
    print("Real and fake transcripts are different distributions.")
    print("Victor might learn something 😧")

stats.plot_comparison(real_samples, fake_samples, "real", "fake")